In [ ]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")


# i. Player data Current Season


In [ ]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 308  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


In [ ]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()
player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])



In [ ]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])

In [ ]:
elements = elements_df.loc[:,['id','team','web_name','first_name','second_name']]
elements = elements.rename(columns = {'id': 'player_id', 'team':'team_id'})

In [ ]:
# List of columns to drop
columns_to_drop = ['bonus', 'bps', 'influence', 'creativity', 'kickoff_time',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out','fixture', 'round']

# Use the drop method to remove the specified columns
player_history_df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
player = player_history_df.rename(columns = {'element': 'player_id'})


In [ ]:
filtered_teams = pd.read_csv('filtered_teams.csv')

In [ ]:
player =  player.merge(elements, on = 'player_id')
player = player.merge(filtered_teams, on = "team_id")

# ii. Analysis